# Praca domowa 3 - Ablation study 
Tematem tej pracy domowej jest zapoznanie się z Ablation study oraz zbadanie wpływu różnych modyfikacje na przykładową sieć.

## Ablation study
 W kontekcie uczenia maszynowego ablation study oznacza modyfikowanie modelu uczącego w celu zbadania wpływu poszczegulnych jego elementów, lub parametrów uczenia na wynik oraz prędkość uczenia. Ablation study pozwala zatem zbadać, które elementy modelu są kluczowe, a które tylko spowalniają uczenie. Umożliwia to lepsze zrozumienie modelu i ewentualne usprawnienie jego działania i być może poprawienia wyniku uczenia.

 source: "https://stats.stackexchange.com/questions/380040/what-is-an-ablation-study-and-is-there-a-systematic-way-to-perform-it" \\
 "https://arxiv.org/abs/1901.08644"

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

W celu przeprowadzenia modyfikacji posłużę się modelem, którego celem będzie rozpoznawanie cyfr ze zbioru MNIST. Do wczytania i przetworzenia danych w celu użycia ich potem w modelu wspomogłem się kodem zamieszczonym na stronie kerasa: "https://keras.io/examples/vision/mnist_convnet/"

In [4]:
batch_size = 32
epochs = 20
num_classes = 10
input_shape = (28, 28, 1)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
def base_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
    model.add(Dropout(0.2))

    model.add(Conv2D(32,(3,3),padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256,activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(128,activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

cnn_n = base_model()
cnn_n.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0

# Sprawdzenie wyników dla początkowej sieci

Na początku trenujemy naszą podstawową sieć, aby sprawdzić jakie wyniki będzie osiągać 

In [6]:
cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test,y_test),shuffle=True, verbose=0)
score = cnn_n.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.024178624153137207
Test accuracy: 0.9929999709129333


Sieć uzyskała zadowalające wyniki. Sprawdzimy teraz jak poradzą sobie zmodyfikowane sieci i czy zmiany te będą miały wpływ na wyniki lub szybkość.

# Sprawdzenie wyników dla sieci bez gęstej warstwy

Sprawdzimy czy usunięcie warstwy dense zmieni wyniki sieci.

In [7]:
def without_dense_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
    model.add(Dropout(0.2))

    model.add(Conv2D(32,(3,3),padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256,activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

cnn_n = without_dense_model()

cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test,y_test),shuffle=True, verbose=0)
score = cnn_n.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.0225150715559721
Test accuracy: 0.9925000071525574


Usunięcie warstwy dense nie wpłynęło w tym przypadku na wynik sieci. Wspomogło to za to delikatnie wydajność, gdyż zmalała ilość parametrów do wytrenowania.

# Sprawdzenie wyników dla sieci bez warstwy konwolucyjnej

Sprawdzimy czy usunięcie warstwy konwolucyjnej zmieni wyniki sieci.

In [8]:
def without_conv_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
    model.add(Dropout(0.2))

    model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(256,activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(128,activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

cnn_n = without_conv_model()
cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test,y_test),shuffle=True, verbose=0)
score = cnn_n.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.030443163588643074
Test accuracy: 0.9901999831199646


W porównaniu z usunięciem warstwy dense otrzymaliśmy znacznie więcej parametrów. Jeśli chodzi o wynik to spadł on bardziej niż w przypadku usunięcia warstwy gęstej. Można wnoskować zatem, że wartwy konwolucyjne mają w tym przypadku większy wpływ na wynik sieci.

## Zmniejszenie neuronów w pierwszej warstwie dense
Najbardziej 'zasobożerną' częścią modelu jest warstwa dens po operacji wypłaszczenia. Sprawdzimy czy zmniejszenie ilości neuronów w tej sieci wpłynie na wynik. Jeżeli wynik się nie zmieni to może okazać się, że możemy zoptymalizować nasz model 

In [9]:
def less_params_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
    model.add(Dropout(0.2))

    model.add(Conv2D(16,(3,3),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(32,activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(32,activation='relu',kernel_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

cnn_n = less_params_model()
cnn_n.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
dropout_14 (Dropout)         (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 16)        4624      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 3136)             

Tym razem nie usuneliśmy żadnej warstwy, natomiast zmodyfikowaliśmy parametry warstw, aby zminimalizować liczbę parametrów do wytrenowania.

In [10]:
cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test,y_test),shuffle=True, verbose=0)
score = cnn_n.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.03733927011489868
Test accuracy: 0.988099992275238


Zmniejszenie liczby parametrów znacząco wpłyneło na wynik modelu. Trenowanie zostało lekko przyspieszone jednak kosztem końcowego wyniku sieci. Sieci też dużo wolniej osiągnełą wysokie wyniki, więc gdybyśmy wzieli mniej epok wynik mógłby być jeszcze gorszy niż dla początkowego modelu z tą samą ilością epok.

## Zmiana batch size 
Na koniec sprawdzimy czy zmiana batch size wpłynie na wynik modelu. Na początku spróbujemy ustwić duży rozmiar paczki

In [13]:
batch_size = 32

cnn_n = base_model()
cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test,y_test),shuffle=True, verbose=0)
score = cnn_n.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.027661358937621117
Test accuracy: 0.9919000267982483


In [14]:
batch_size = 256

cnn_n = base_model()
cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test,y_test),shuffle=True, verbose=0)
score = cnn_n.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.021962296217679977
Test accuracy: 0.9940000176429749


Batch size w tym przypadku nie robi dużej różnicy, jednak delikatnie lepsze wyniki osiąga zazwyczaj większy batch size

Powyższe eksperymenty pozwoliły nam sprawdzić, które elementy sieci są dla nas kluczowe, a które być może nawet niepotrzebne. W naszym przypadku większość zmian nie wpłyneła znacząco na wyniki. Tylko znaczące zredukowanie ilości neuronów spowodowało większy spadek dokładności. W pozostałych przypadkach różnice były nieduże. Nasza sieć zapewne była na tyle mocna w porównaniu do stosunkwo prostego zbioru MNIST, że po zmianach dalej świetnie sobie radziła. 